In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
from glob import glob

import numpy as np
import pandas as pd

In [8]:
from xray import data, params, trainer, utils

In [9]:
filename = "xray_df.csv"

In [17]:
df = data.get_data_from_gcp(filename)
df.head(3)

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],...,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,Fixed_Labels,path,cat_col
0,00000001_000.png,0,1,58,M,PA,2682,2749,0.143,0.143,...,0,0,0,0,0,0,0,Cardiomegaly,/content/drive/MyDrive/Proyecto_Lewagon_Rayos_...,['Cardiomegaly']
1,00000001_001.png,1,1,58,M,PA,2894,2729,0.143,0.143,...,0,0,0,0,0,0,0,Cardiomegaly|Emphysema,/content/drive/MyDrive/Proyecto_Lewagon_Rayos_...,"['Cardiomegaly', 'Emphysema']"
2,00000001_002.png,2,1,58,M,PA,2500,2048,0.168,0.168,...,0,0,0,0,0,0,0,Cardiomegaly|Effusion,/content/drive/MyDrive/Proyecto_Lewagon_Rayos_...,"['Cardiomegaly', 'Effusion']"


In [22]:
df.drop(
    columns=[
        "Follow-up #",
        "Patient Age",
        "Patient Gender",
        "View Position",
        "OriginalImagePixelSpacing_x",
        "OriginalImagePixelSpacing_y",
        "OriginalImageWidth",
        "OriginalImageHeight",
        "OriginalImage[Width",
        "Height]",
        "OriginalImagePixelSpacing[x",
        "y]",
        "Count_diseases",
    ],
    inplace=True,
    errors="ignore",
)

In [24]:
df['path'] = df.path.map(lambda x: "/".join(x.split("/")[-3:]))

In [25]:
df.head(3)

,Image Index,Patient ID,Enfermo,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,...,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,Fixed_Labels,path,cat_col
0,00000001_000.png,1,True,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,Cardiomegaly,images_001/images/00000001_000.png,['Cardiomegaly']
1,00000001_001.png,1,True,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,Cardiomegaly|Emphysema,images_001/images/00000001_001.png,"['Cardiomegaly', 'Emphysema']"
2,00000001_002.png,1,True,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,Cardiomegaly|Effusion,images_001/images/00000001_002.png,"['Cardiomegaly', 'Effusion']"


In [29]:
df_sick = df[df["Enfermo"] != False]
df_sick.shape

(51751, 21)

In [30]:
ds_train, ds_val, ds_test = data.split_df(
    dataset=df_sick,
    column_to_filter_by="Patient ID",
    train_val_test=(0.65, 0.15, 0.15),
    total_filter=1,
)
print("train :", ds_train.shape)
print("val :", ds_val.shape)
print("test :", ds_test.shape)

train : (33411, 21)
val : (8241, 21)
test : (7406, 21)


In [31]:
input_shape = (224, 224)

In [34]:
filename

'xray_df.csv'

In [38]:
path_to_csv = params.GCP_IMAGE_BUCKET
path_to_csv

'gs://xray-lewagon-testupload/images/'

In [37]:
gen_train = data.build_generator(
    img_path=path_to_csv,
    labels_df=ds_train,
    index_col="path",
    labels_col="Fixed_Labels",
    train_set_batch_size=32,
    target_size=(input_shape[0], input_shape[1]),
    binary_class=False,
    test_set=False,
    data_augment=False,
)

gen_val = data.build_generator(
    img_path=path_to_csv,
    labels_df=ds_val,
    index_col="path",
    labels_col="Fixed_Labels",
    train_set_batch_size=32,
    target_size=(input_shape[0], input_shape[1]),
    binary_class=False,
    test_set=False,
    data_augment=False,
)

gen_test = data.build_generator(
    img_path=path_to_csv,
    labels_df=ds_test,
    index_col="path",
    labels_col="Fixed_Labels",
    train_set_batch_size=1,
    target_size=(input_shape[0], input_shape[1]),
    binary_class=False,
    test_set=True,
    data_augment=False,
)


Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/home/santiago/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 33411 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(
/home/santiago/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 8241 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(
/home/santiago/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/keras_preprocessing/image/dataframe_iterator.py:279: UserWarning: Found 7406 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(
